# Contexte

Ce notebook est la deuxième partie d'un projet ou nous devons fournir des analyses concernant la répartition de l'alimentation, la sous-nutrition mais également en comprendre les causes.

# Sommaire

* [Préparation du dataset](#chapter3)
    
    
* [Analyse](#chapter4)
    * [Manioc](#section_4_1)
    * [Aide alimentaire](#section_4_2)
    * [Répartition Humain/Animale](#section_4_3)

# Préparation du dataset
<a class="anchor" id="chapter3"></a>

In [1]:
#import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno

import plotly.express as px

pd.set_option('display.max_columns', None)

In [2]:
#import des données
aide = pd.read_csv("data/aide_alimentaire.csv", delimiter=",")
dispo = pd.read_csv("data/dispo_alimentaire.csv", delimiter=",")
pop = pd.read_csv("data/population.csv", delimiter=",")
inséc = pd.read_csv("data/sous_nutrition.csv", delimiter=",")

In [3]:
#changement des noms de colonnes dans les datasets
aide1 = aide.rename(columns={"Pays bénéficiaire": "Zone","Valeur": "prod_données_tonne","Produit":"prod_données"})
pop1 = pop.rename(columns={"Valeur": "population"})
inséc1 = inséc.rename(columns={"Valeur": "mean_und-alim_million"})

#remplacement des tranches d'années
inséc1 = inséc1.replace(['2012-2014', '2013-2015', '2014-2016', '2015-2017','2016-2018','2017-2019'],
                        ['2013', '2014', '2015', '2016','2017','2018'])

#changement du type
inséc1['Année'] = inséc1['Année'].astype('int64')

In [4]:
#jointure de toute les datasets
df = pd.merge(pd.merge(pd.merge(aide1,pop1,on=['Zone',"Année"],how='outer'),inséc1,on=['Zone',"Année"],how='outer'),dispo,on='Zone')

In [5]:
#imputationdes valeur <0.1 par 0
df['mean_und-alim_million'] = df['mean_und-alim_million'].replace({"<0.1": 0})

In [6]:
#changement du type de la colonne
df['mean_und-alim_million'] = df['mean_und-alim_million'].astype('float')

In [8]:
#filtrage pour l'année 2017
df_2017 = df[df['Année'] == 2017]

# Analyses
<a class="anchor" id="chapter4"></a>

## Manioc
<a class="anchor" id="section_4_1"></a>

In [9]:
#selection du manioc et de la thaïlande
thai = df_2017[df_2017['Produit'] == 'Manioc']
thai = thai[thai['Zone'] == 'Thaïlande']

In [10]:
#passage de la population de millier en million
thai['population'] = thai['population']*1e-3

In [11]:
thai

,Zone,Année,prod_données,prod_données_tonne,population,mean_und-alim_million,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
183588,Thaïlande,2017,NaN,NaN,69.20981,6.2,Manioc,vegetale,1800.0,2081.0,40.0,13.0,0.05,0.14,6264.0,25214.0,1250.0,871.0,1511.0,30228.0,NaN,0.0,0.0


In [12]:
#calcul de la proportion de sous_nutrition
thai['proportion_under'] = thai['mean_und-alim_million']/thai['population']*100

In [13]:
#calcul du taux d'export par la production
thai['proportion_export'] = thai['Exportations - Quantité']/thai['Production']*100

In [14]:
#affichage plus beau
thai = thai[['population','mean_und-alim_million','Produit','Exportations - Quantité','proportion_under','proportion_export']]

In [15]:
thai

,population,mean_und-alim_million,Produit,Exportations - Quantité,proportion_under,proportion_export
183588,69.20981,6.2,Manioc,25214.0,8.958268,83.41273


## Aide alimentaire
<a class="anchor" id="section_4_2"></a>

In [16]:
#groupby par pays sur la df de base merged
aide = df.groupby(['Zone']).sum()

In [17]:
#selection des aides
aide = aide[['prod_données_tonne']]

In [18]:
#triage et affichage
aide.sort_values(by=['prod_données_tonne'], ascending= False).head(10)

,prod_données_tonne
Zone,
Éthiopie,127079048.0
Yémen,112203012.0
Soudan,56261856.0
Kenya,53072256.0
Bangladesh,32729672.0
Niger,25147304.0
Pakistan,21720768.0
Tchad,19561518.0
République-Unie de Tanzanie,16514112.0


## Répartition Humain/Animale
<a class="anchor" id="section_4_3"></a>

In [92]:
#filtrage par céréales
repart = df_2017[df_2017['Produit'].isin(['Blé','Céréales, Autres','Soja','Seigle','Sorgho','Riz (Eq Blanchi)','Orge','Maïs','Millet'])]

In [93]:
#~selection des colonnes
repart = repart[['Zone','Aliments pour animaux','Nourriture','Disponibilité intérieure']]

In [94]:
#groupby par pays
repartition = repart.groupby(['Zone']).sum()

In [95]:
#calcul des proportions humaines et animales par la disponibilité 
repartition['proportion_animale'] = repartition['Aliments pour animaux']/repartition['Disponibilité intérieure']*100
repartition['proportion_humain'] = repartition['Nourriture']/repartition['Disponibilité intérieure']*100

In [96]:
repartition

,Aliments pour animaux,Nourriture,Disponibilité intérieure,proportion_animale,proportion_humain
Zone,,,,,
Afghanistan,560.0,5494.0,7303.0,7.668082,75.229358
Afrique du Sud,4824.0,9530.0,16176.0,29.821958,58.914441
Albanie,386.0,474.0,1117.0,34.556849,42.435094
Algérie,4230.0,8535.0,14891.0,28.406420,57.316500
Allemagne,26043.0,9136.0,44786.0,58.149868,20.399232
...,...,...,...,...,...
Émirats arabes unis,1055.0,1485.0,3304.0,31.930993,44.945521
Équateur,847.0,1458.0,3447.0,24.572092,42.297650
États-Unis d'Amérique,140906.0,32557.0,392881.0,35.864804,8.286733


In [97]:
somme_animal = repartition['Aliments pour animaux'].sum() / repartition['Disponibilité intérieure'].sum() * 100
somme_hunimal = repartition['Nourriture'].sum() / repartition['Disponibilité intérieure'].sum() * 100

In [98]:
print(somme_animal)
print(somme_hunimal)

32.838224303341825
39.18524930219789
